In [3]:
### Run this cell before continuing.
  
    

*STAT 201 - GROUP 7*

# (Title)

# Introduction

### Background

### Question

# Preliminary Results

##### Read, Clean, & Wrangle

##### Plot(s)

##### Table of Estimates

# Methods: Plan

(The previous sections will carry over to your final report (you’ll be allowed to improve them based on feedback you get). Begin this Methods section with a brief description of “the good things” about this report – specifically, in what ways is this report trustworthy?

Continue by explaining why the plot(s) and estimates that you produced are not enough to give to a stakeholder, and what you should provide in addition to address this gap. Make sure your plans include at least one hypothesis test and one confidence interval. If possible, compare both the bootstrapping and asymptotics methods.)



##### What do you expect to find?

 ##### What impact could such findings have?

##### What future questions could this lead to?

# References

At least two citations of literature relevant to the project. The citation format is your choice – just be consistent. Make sure to cite the source of your data as well.

- 1
- 2
